In [12]:
import numpy as np
import pandas as pd
import copy
from gensim.models import Word2Vec
import keras
from numpy import dot
import math
from sklearn.preprocessing import normalize
from keras.utils import np_utils
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
df = pd.read_csv('./all_after_mapping.tsv',sep='\t',names=column_names)

labels = df['type'].values
labels = np.array(labels)
print(labels)
# labels = np_utils.to_categorical(labels)
# print(labels.shape)

column_names = ['type','title','text']
df_train = pd.read_csv('./data_after_sep/train.tsv',sep='\t',names=column_names)
df_test = pd.read_csv('./data_after_sep/test.tsv',sep='\t',names=column_names)
df_dev = pd.read_csv('./data_after_sep/dev.tsv',sep='\t',names=column_names)

[0 0 2 ... 3 6 2]


In [34]:
tokenlizeword = np.load('tokenlizeword0225_nopunct.npy',allow_pickle=True)
dev_x = tokenlizeword[:5000]
dev_y = labels[:5000]

test_x = tokenlizeword[5000:10000]
test_y = labels[5000:10000]

train_x = tokenlizeword[10000:]
train_y = labels[10000:]

In [35]:
count = 0
for k in range(len(train_x)):
    train_x[k] = np.array(train_x[k])
    count+=train_x[k].shape[0]
##存各類分開
li_0 = []
li_1 = []
li_2 = []
li_3 = []
li_4 = []
li_5 = []
li_6 = []
for k in range(len(train_x)):
    if train_y[k]==0:
        li_0.append(' '.join(train_x[k]))
    elif train_y[k]==1:
        li_1.append(' '.join(train_x[k]))
    elif train_y[k]==2:
        li_2.append(' '.join(train_x[k]))
    elif train_y[k]==3:
        li_3.append(' '.join(train_x[k]))
    elif train_y[k]==4:
        li_4.append(' '.join(train_x[k]))
    elif train_y[k]==5:
        li_5.append(' '.join(train_x[k]))
    elif train_y[k]==6:
        li_6.append(' '.join(train_x[k]))
li_0 = np.array(li_0) ## input
li_1 = np.array(li_1) ## input
li_2 = np.array(li_2) ## input
li_3 = np.array(li_3) ## input
li_4 = np.array(li_4) ## input
li_5 = np.array(li_5) ## input
li_6 = np.array(li_6) ## input


6361


In [43]:
## 全部搞成tfidf
vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words=None, token_pattern="(?u)\\b\\w+\\b", smooth_idf=True, norm='l2')
tfidf_0 = vectorizer.fit_transform(li_0)
df_tfidf_0 = pd.DataFrame(tfidf_0.toarray(),columns=vectorizer.get_feature_names())

tfidf_1 = vectorizer.fit_transform(li_1)
df_tfidf_1 = pd.DataFrame(tfidf_1.toarray(),columns=vectorizer.get_feature_names())

tfidf_2 = vectorizer.fit_transform(li_2)
df_tfidf_2 = pd.DataFrame(tfidf_2.toarray(),columns=vectorizer.get_feature_names())

tfidf_3 = vectorizer.fit_transform(li_3)
df_tfidf_3 = pd.DataFrame(tfidf_3.toarray(),columns=vectorizer.get_feature_names())

tfidf_4 = vectorizer.fit_transform(li_4)
df_tfidf_4 = pd.DataFrame(tfidf_4.toarray(),columns=vectorizer.get_feature_names())

tfidf_5 = vectorizer.fit_transform(li_5)
df_tfidf_5 = pd.DataFrame(tfidf_5.toarray(),columns=vectorizer.get_feature_names())

tfidf_6 = vectorizer.fit_transform(li_6)
df_tfidf_6 = pd.DataFrame(tfidf_6.toarray(),columns=vectorizer.get_feature_names())

In [44]:
map_0 = df_tfidf_0.describe()
map_1 = df_tfidf_1.describe()
map_2 = df_tfidf_2.describe()
map_3 = df_tfidf_3.describe()
map_4 = df_tfidf_4.describe()
map_5 = df_tfidf_5.describe()
map_6 = df_tfidf_6.describe()

In [45]:

map_list = []
map_list.append(map_0)
map_list.append(map_1)
map_list.append(map_2)
map_list.append(map_3)
map_list.append(map_4)
map_list.append(map_5)
map_list.append(map_6)


In [61]:
pred_list = []
NUM_LABELS = 7
##看看train的資料的
for i in range(len(train_x)):
    score_list = np.zeros((NUM_LABELS))
    for m in range(NUM_LABELS):
        score_list[m] = 1.0
    result_list = np.zeros((NUM_LABELS))
    num = 0.0
    print(train_y[i])
    for j in range(len(train_x[i])):
        for k in range(NUM_LABELS):
            if train_x[i][j] not in map_list[k]:
                score_list[k] += 0
            else:
                if(map_list[k].iloc[1][train_x[i][j]]>=0.015): ##閥值
                    score_list[k] *= (1+map_list[k].iloc[1][train_x[i][j]])
    print(score_list)
    for k in range(NUM_LABELS):
        num += score_list[k]
    for k in range(NUM_LABELS):
        result_list[k] = score_list[k]/num ##除以和
    pred_list.append(result_list)

0
[4.9687818  2.03975728 3.53408197 4.31450146 3.91208031 2.7029112
 2.42537618]
6
[5.75580836 3.2632302  7.37393868 7.83089807 6.40426534 3.82603156
 6.55366783]
4
[1.26847062 1.14321458 1.30468732 1.32422111 1.31694737 1.17682618
 1.20128547]
2
[2.65008331 1.57441942 3.46755844 2.94340177 2.68528719 1.96381353
 2.55500042]
0
[1.90135531 1.3030789  1.61261697 2.04749711 1.75781504 1.39178151
 1.60735024]
3
[1.13510041 1.08230452 1.21291199 1.24312126 1.09522847 1.16063919
 1.13982962]
5
[3.27258217 1.80863941 2.67449296 4.82338105 4.18350907 2.52978425
 2.45343318]
3
[4.44956226 2.27765797 4.16715194 5.5677678  4.5434646  2.84539133
 3.13897527]
6
[3.4892802  2.73214542 6.52661    4.57348929 3.36483661 2.85692448
 6.90050277]
1
[2.60489044 1.88396539 2.33506214 2.64661894 2.37157218 1.88358791
 2.22792713]
6
[4.04565558 2.96062433 7.23349732 6.56407972 3.95549433 2.96318915
 7.86860439]
1
[2.3334555  1.80914902 2.46617698 2.88973431 2.58304775 1.88093207
 2.09903463]
0
[3.43936276 1.6

[1.69893504 1.27261342 1.44117298 1.3824133  1.37164373 1.2899014
 1.36043605]
0
[5.12125139 3.0714752  5.58398085 6.78041452 5.85553183 3.82871763
 4.42458606]
5
[1.32459731 1.31051204 1.53833844 1.42935668 1.35413071 1.40904418
 1.41722187]
0
[1.55137442 1.3614031  1.61369186 1.38544132 1.27973704 1.3437857
 1.66620697]
1
[1.29455021 1.32067389 1.36459447 1.5417768  1.29146935 1.34965472
 1.59606934]
2
[2.92400225 2.14346951 5.20968652 3.57110659 2.54026803 2.26802442
 4.82943793]
2
[1.73658812 1.47198575 2.15593772 1.80061994 1.72288064 1.6913868
 1.74865024]
5
[3.0128602  2.27437733 3.80735935 2.89721108 2.87063248 2.88920131
 2.61513447]
1
[1.22447107 1.19604251 1.34156561 1.24353069 1.14107898 1.19236487
 1.43229822]
4
[1.16467799 1.16066125 1.24841339 1.21656533 1.16411258 1.31036029
 1.45765942]
1
[2.06145124 2.04647559 2.25677546 2.5232911  2.15804185 2.25318641
 2.37126414]
1
[4.71681527 3.3116471  5.05662095 7.41641789 4.9451458  3.26573909
 4.17557796]
4
[1.77705015 1.53667

KeyboardInterrupt: 

In [52]:
pred_list = np.array(pred_list)
pred = np.argmax(pred_list, axis=1)
print(pred)
print(train_y)
# correct += (pred == labels).sum().item()

# print(correct/len(train_x))

[0 6 4 ... 3 6 2]
[0 6 4 ... 3 6 2]


In [53]:
count = 0
for i in range(len(train_y)):
    if(pred[i]==train_y[i]):
        count += 1
        
print(count/len(train_y))

0.7010099428481954
